### Define Data Quality KPIs

**Task 1**: Identify Relevant KPIs

**Objective**: Develop KPIs that align with organizational goals.

**Steps**:
1. Choose a dataset from a domain of your interest (e.g., sales data, healthcare records, or transaction logs).
2. Identify three KPIs that would be crucial for assessing the data quality in your chosen dataset. Consider accuracy, completeness, and timeliness.
3. Document why each KPI is important for maintaining high-quality data in your given context.

In [1]:
# Task 1: Identify Relevant KPIs for Sales Data

import pandas as pd
import numpy as np
import datetime as dt

# Sample Sales Dataset
data = {
    'Order_ID': [101, 102, 103, 104, 104],
    'Customer_Name': ['Alice', 'Bob', 'Charlie', None, 'Eve'],
    'Order_Date': ['2023-01-10', '2023-01-15', '2023-01-20', '2023-01-25', None],
    'Amount': [250, 300, 400, None, 200]
}

df = pd.DataFrame(data)
df['Order_Date'] = pd.to_datetime(df['Order_Date'], errors='coerce')

# KPI 1: Completeness (Percentage of non-null values)
def kpi_completeness(df):
    return (df.notnull().sum() / len(df) * 100).round(2)

# KPI 2: Accuracy (simulate with known valid order IDs)
def kpi_accuracy(df, column, reference_list):
    valid = df[column].isin(reference_list).sum()
    total = df[column].notnull().sum()
    return round(100 * valid / total, 2) if total > 0 else 0

# KPI 3: Timeliness (e.g., how recent the last order is)
def kpi_timeliness(df, date_column):
    today = pd.Timestamp.today()
    most_recent = df[date_column].max()
    days_diff = (today - most_recent).days if pd.notnull(most_recent) else np.nan
    return f"{days_diff} days since last entry"

# Display KPI Results
print("=== Data Quality KPIs for Sales Dataset ===")
print("\nKPI 1 - Completeness (%):")
print(kpi_completeness(df))

print(f"\nKPI 2 - Accuracy of 'Order_ID': {kpi_accuracy(df, 'Order_ID', [101, 102, 103, 104])}%")

print(f"\nKPI 3 - Timeliness of 'Order_Date': {kpi_timeliness(df, 'Order_Date')}")


=== Data Quality KPIs for Sales Dataset ===

KPI 1 - Completeness (%):
Order_ID         100.0
Customer_Name     80.0
Order_Date        80.0
Amount            80.0
dtype: float64

KPI 2 - Accuracy of 'Order_ID': 100.0%

KPI 3 - Timeliness of 'Order_Date': 846 days since last entry


**Task 2**: Develop a KPI Dashboard

**Objective**: Visualize your KPIs for better monitoring.

**Steps**:
1. Use a tool like Excel or a BI tool (e.g., Tableau, Power BI) to create a simple dashboard.
2. Input sample data and visualize your chosen KPIs, showing how they would be monitored.
3. Share your dashboard with peers and gather feedback on KPI relevance and clarity.

In [3]:
import pandas as pd
import dash
from dash import html, dcc
import plotly.graph_objs as go

# Sample sales data
data = {
    'CustomerID': [1, 2, 3, 4, None],
    'Email': ['john@example.com', 'bob@example.com', None, 'alice@example.com', 'eve@example.com'],
    'PurchaseDate': ['2023-04-01', '2023-04-02', '2023-01-15', '2023-03-28', None],
    'Amount': [100, 200, None, 300, 150]
}

df = pd.DataFrame(data)

# Calculate KPI metrics
def calculate_accuracy(df, column, valid_values):
    correct = df[column].isin(valid_values).sum()
    total = df[column].notna().sum()
    return (correct / total) * 100 if total > 0 else 0

def calculate_completeness(df, column):
    filled = df[column].notna().sum()
    total = len(df[column])
    return (filled / total) * 100 if total > 0 else 0

def calculate_timeliness(df, date_column, cutoff_date):
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    recent = df[date_column] >= pd.to_datetime(cutoff_date)
    timely = recent.sum()
    total = df[date_column].notna().sum()
    return (timely / total) * 100 if total > 0 else 0

# KPI Calculations
accuracy_email = calculate_accuracy(df, 'Email', [
    'john@example.com', 'bob@example.com', 'alice@example.com', 'david@example.com', 'eve@example.com'
])
completeness_customer = calculate_completeness(df, 'CustomerID')
timeliness_purchase = calculate_timeliness(df, 'PurchaseDate', '2023-03-01')

# Create Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Sales Data Quality KPI Dashboard", style={'textAlign': 'center'}),

    html.Div([
        dcc.Graph(
            figure=go.Figure(go.Indicator(
                mode="gauge+number",
                value=accuracy_email,
                title={'text': "Email Accuracy (%)"},
                gauge={'axis': {'range': [0, 100]}}
            ))
        ),
        dcc.Graph(
            figure=go.Figure(go.Indicator(
                mode="gauge+number",
                value=completeness_customer,
                title={'text': "Customer ID Completeness (%)"},
                gauge={'axis': {'range': [0, 100]}}
            ))
        ),
        dcc.Graph(
            figure=go.Figure(go.Indicator(
                mode="gauge+number",
                value=timeliness_purchase,
                title={'text': "Purchase Date Timeliness (%)"},
                gauge={'axis': {'range': [0, 100]}}
            ))
        )
    ], style={'display': 'flex', 'justifyContent': 'space-around'})
])

if __name__ == '__main__':
    app.run(debug=True)
